# Lasso para inferencia

En este laboratorio exploramos la capacidad de doble Lasso para estimar el parámetro de interés, a través del procedimiento de *partialling-out*. Usaremos una muestra restringida de los datos de la GEIH para Barranquilla AM, a fin de generar un escenario de alta dimensionalidad, $p>n$

## Brecha salarial

Queremos responder la siguiente pregunta ¿Cuál es la diferencia de salarios entre hombres y mujeres con las mismas caraceterísticas observables?

\begin{equation*}
logy=\alpha D+\boldsymbol{\beta}'W+\epsilon
\end{equation*}

donde $Y$ es el salario por hora, $D$, es la dummy que identifica a las mujeres, y $W$ es un vector de características observables así como de regresores técnicos. En W incluimos los años de experiencia, el nivel educativo, el sector económico, la ocupación, así como todas interacciones entre las variables. 

### Cargar y definir la muestra

In [36]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
import sys
from sklearn.base import BaseEstimator
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

In [ ]:

file = "https://raw.githubusercontent.com/andvarga-eco/econometrics_master_uninorte/refs/heads/main/Datos%20y%20c%C3%B3digo/geih_24_baq.csv"
df = pd.read_csv(file)
# Filter df by MES == 1
print(df.shape)

#Construir variables

df=df[(df['P6040']>24)&((df['P6040']<65))]

df['lwage']=np.log(df['INGLABO']/df['P6800'])
df['exp']=df['P6040']-16
df['exp2']=df['exp']^2
df['exp3']=df['exp']^3
df['exp4']=df['exp']^4

conditions=[
    df['P3042']<3,
    (df['P3042']>=3) & (df['P3042']<8),
    (df['P3042']>=8) & (df['P3042']<11),
    (df['P3042']>=11) & (df['P3042']<14),
]
choices=[1,2,3,4]

df['educ']=np.select(conditions,choices)
df['educ']=df['educ'].astype('category')


df['P3271']=df['P3271'].astype('category')


# Create 'sex' variable: 1 if P3271==2, 0 if P3271==1
df['sex'] = np.where(df['P3271'] == 2, 1, np.where(df['P3271'] == 1, 0, np.nan))


df['sector'] = df['RAMA2D_R4'].astype(str).str[:1].astype(int)
df['sector'].value_counts()
df['occ'] = df['OFICIO_C8'].astype(str).str[:1].astype(int)


df.describe()


(5167, 18)


,PERIODO,MES,DIRECTORIO,SECUENCIA_P,ORDEN,INGLABO,P6500,P6800,RAMA2D_R4,RAMA4D_R4,...,AREA,CLASE,lwage,exp,exp2,exp3,exp4,sex,sector,occ
count,4.373000e+03,4373.000000,4.373000e+03,4373.000000,4373.000000,4.329000e+03,4.329000e+03,4373.000000,4373.000000,4373.000000,...,4373.0,4373.0,4329.000000,4373.000000,4373.000000,4373.000000,4373.000000,4373.000000,4373.000000,4373.000000
mean,2.024064e+07,6.103362,7.839283e+06,1.008004,2.095815,2.114515e+06,2.076537e+06,48.241024,54.238052,5451.285159,...,8.0,1.0,10.469856,23.939172,23.980791,23.978276,24.022410,0.410016,4.987651,4.821633
std,3.643551e+02,3.490979,1.278284e+05,0.109811,1.538828,2.186301e+06,2.126770e+06,8.124263,23.194182,2318.375329,...,0.0,0.0,0.616292,10.219277,10.428276,10.501241,10.552247,0.491892,2.247461,2.537409
min,2.024010e+07,1.000000,7.656021e+06,1.000000,1.000000,5.000000e+04,5.000000e+04,6.000000,1.000000,111.000000,...,8.0,1.0,7.824046,9.000000,8.000000,8.000000,8.000000,0.000000,1.000000,1.000000
25%,2.024031e+07,3.000000,7.729140e+06,1.000000,1.000000,1.300000e+06,1.300000e+06,48.000000,42.000000,4210.000000,...,8.0,1.0,10.206674,15.000000,15.000000,15.000000,15.000000,0.000000,4.000000,3.000000
50%,2.024062e+07,6.000000,7.800470e+06,1.000000,2.000000,1.460000e+06,1.400000e+06,48.000000,49.000000,4921.000000,...,8.0,1.0,10.322746,23.000000,23.000000,23.000000,23.000000,0.000000,4.000000,5.000000
75%,2.024104e+07,10.000000,7.981937e+06,1.000000,3.000000,2.000000e+06,2.000000e+06,48.000000,77.000000,7730.000000,...,8.0,1.0,10.691945,31.000000,31.000000,31.000000,31.000000,1.000000,7.000000,7.000000
max,2.024125e+07,12.000000,8.086540e+06,4.000000,22.000000,3.500000e+07,3.500000e+07,120.000000,99.000000,9900.000000,...,8.0,1.0,13.520711,48.000000,50.000000,51.000000,52.000000,1.000000,9.000000,9.000000


In [19]:
# Muestra aleatoria, n=200
df_sample = df.sample(n=200, random_state=42)  # random_state for reproducibility
print(f"Original df shape: {df.shape}")
print(f"Sample df shape: {df_sample.shape}")

Original df shape: (4373, 27)
Sample df shape: (200, 27)


### Estimación por MCO

In [40]:
# Modelo extraflexbile, p>200
model_extra=('lwage~sex+(exp+exp2+exp3+exp4+C(educ)+C(sector)+C(occ))**2')
model_mco= smf.ols(model_extra, data=df_sample).fit()
sex_est=model_mco.params['sex']
sex_se=model_mco.HC3_se['sex']
model_mcoR2=model_mco.rsquared
model_mcoMSE=model_mco.mse_resid
n_params = len(model_mco.params)

print(f"The estimated sex coefficient is {sex_est:.4f} "
      f"and the corresponding robust standard error is {sex_se:.4f}")
print(f"The model R-squared is {model_mcoR2:.4f} "
      f"and the model MSE is {model_mcoMSE:.4f}")
print(f"Number of parameters estimated: {n_params}")


The estimated sex coefficient is 0.0442 and the corresponding robust standard error is 0.2245
The model R-squared is 0.8587 and the model MSE is 0.1929
Number of parameters estimated: 203


### Estimación por Lasso: *partialling-out**

In [33]:
import hdmpy

# We wrap the package so that it has the familiar sklearn API
class RLasso(BaseEstimator):

    def __init__(self, *, post=True):
        self.post = post

    def fit(self, X, y):
        self.rlasso_ = hdmpy.rlasso(X, y, post=self.post)
        return self

    def predict(self, X):
        pred = np.array(X) @ np.array(self.rlasso_.est['beta']).flatten()
        pred += np.array(self.rlasso_.est['intercept'])
        return pred

    def nsel(self):
        return sum(abs(np.array(self.rlasso_.est['beta']).flatten() > 0))


def lasso_model():
    return RLasso(post=False)

In [28]:
#Modelos
## Y sobre W
model_yW='lwage~(exp+exp2+exp3+exp4+C(educ)+C(sector)+C(occ))**2'
X_y=smf.ols(model_yW, data=df_sample).data.exog[:,1:]
y_y=smf.ols(model_yW, data=df_sample).data.endog
## Sex sobre W
model_sexW='sex~(exp+exp2+exp3+exp4+C(educ)+C(sector)+C(occ))**2'
X_sex=smf.ols(model_sexW, data=df_sample).data.exog[:,1:]
y_sex=smf.ols(model_sexW, data=df_sample).data.endog

In [38]:
# Partialling out el efecto de W sobre Y

t_y_lasso=y_y-lasso_model().fit(X_y, y_y).predict(X_y)

# Partialling out el efecto de W sobre sex
t_sex_lasso=y_sex-lasso_model().fit(X_sex, y_sex).predict(X_sex)

# Regresión de Y sobre sex después de partialling out
partial_lasso=sm.OLS(t_y_lasso, sm.add_constant(t_sex_lasso)).fit()
partial_lasso_est = partial_lasso.params[1]

print("Coefficient for sex via partialling-out using lasso " + str(partial_lasso_est))

# standard error
partial_lasso_se = partial_lasso.HC3_se[1]
print("Standard error: " + str(partial_lasso_se))

# confidence interval
print("95% CI: " + str(partial_lasso.conf_int()[1]))



Coefficient for sex via partialling-out using lasso -0.023261750366233624
Standard error: 0.0791381193037471
95% CI: [-0.1652503  0.1187268]


In [39]:
table2 = pd.DataFrame()

table2['Model'] = ["MCO", "Lasso"]

table2['Estimate'] = [sex_est, partial_lasso_est]

table2['Std. Error'] = [sex_se, partial_lasso_se]

# Show results
table2

,Model,Estimate,Std. Error
0,MCO,0.044194,0.224535
1,Lasso,-0.023262,0.079138
